# Many node graph (7 nodes)
## Still using `sqlite` dialect

In [1]:
import pandas as pd
import sqlite3
import json
import os
import typing


# examples for using SQL engines and dialects
from graphreduce.node import SQLNode
from graphreduce.graph_reduce import GraphReduce
from graphreduce.enum import SQLOpType, ComputeLayerEnum
from graphreduce.models import sqlop

/Users/wesmadrigal/projects/graphreduce/venv/lib/python3.9/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [2]:
!ls dat/

cust.csv                      order_events.csv
cust.db                       order_products.csv
notification_interactions.csv orders.csv
notifications.csv             products.csv


In [35]:
!rm dat/cust.db

In [36]:
dbfile = 'dat/cust.db'
conn = sqlite3.connect(dbfile)

In [37]:
files = [x for x in os.listdir('dat/') if x.endswith('.csv')]
# add each file to the database
for f in files:
    df = pd.read_csv(f"dat/{f}")
    name = f.split('.')[0]
    df.to_sql(name, conn, if_exists='replace', index=False)

In [38]:
class CustNode(SQLNode):
    def do_annotate(self) -> typing.Union[sqlop, typing.List[sqlop]]:
        return [
            sqlop(optype=SQLOpType.select, opval=f"*, LENGTH({self.colabbr('name')}) as {self.colabbr('name_length')}")
        ]
    
    def do_filters(self) -> typing.Union[sqlop, typing.List[sqlop]]:
        return [
            sqlop(optype=SQLOpType.where, opval=f"{self.colabbr('id')} < 3")
        ]
    
    def do_normalize(self):
        pass
    
    
    def do_reduce(self, reduce_key):
        pass
    
    def do_post_join_annotate(self):
        pass
    
    def do_post_join_filters(self):
        pass

In [39]:
class NotificationNode(SQLNode):
    def do_annotate(self) -> typing.List[sqlop]:
        return [
            sqlop(optype=SQLOpType.select, opval=f"*, strftime('%m', {self.colabbr('ts')})")
        ]
    
    def do_filters(self) -> typing.List[sqlop]:
        return [
            sqlop(optype=SQLOpType.where, opval=f"{self.colabbr('ts')} > '2022-06-01'")
        ]
    
    def do_normalize(self):
        pass
    
    def do_reduce(self, reduce_key):
        return [
            # Shouldn't this just be a select?
            sqlop(optype=SQLOpType.aggfunc, opval=f"count(*) as {self.colabbr('num_notifications')}"),
            sqlop(optype=SQLOpType.agg, opval=f"{self.colabbr(reduce_key)}")
        ]
    

In [40]:
!cat dat/orders.csv

id,customer_id,ts,amount
1,1,2023-05-12,10
2,1,2023-06-01,11.5
3,2,2023-01-01,100
4,2,2022-08-05,150
5,1,2023-07-01,325
6,2,2023-07-02,23
7,1,2023-07-14,12000


In [41]:
class OrderNode(SQLNode):
    def do_filters(self) -> typing.List[sqlop]:
        return [
            sqlop(optype=SQLOpType.where, opval=f"{self.colabbr(self.date_key)} > '2022-12-01'")
        ]
    
    def do_annotate(self) -> typing.List[sqlop]:
        pass
    
    def do_normalize(self):
        pass
    
    def do_reduce(self, reduce_key):
        return [
            # Shouldn't this just be a select?
            sqlop(optype=SQLOpType.aggfunc, opval=f"count(*) as {self.colabbr('num_orders')}"),
            sqlop(optype=SQLOpType.agg, opval=f"{self.colabbr(reduce_key)}")
        ]
    
    
    def do_labels(self, reduce_key):
        return [
            sqlop(optype=SQLOpType.aggfunc, opval=f"count(*) as {self.colabbr('num_orders_label')}"),
            sqlop(optype=SQLOpType.agg, opval=f"{self.colabbr(reduce_key)}")
        ]

In [42]:
!ls dat/

cust.csv                      order_events.csv
cust.db                       order_products.csv
notification_interactions.csv orders.csv
notifications.csv             products.csv


In [43]:
class OrderEventNode(SQLNode):
    
    def do_reduce(self, reduce_key):
        return [
            # Shouldn't this just be a select?
            sqlop(optype=SQLOpType.aggfunc, opval=f"count(*) as {self.colabbr('num_order_events')}"),
            sqlop(optype=SQLOpType.agg, opval=f"{self.colabbr(reduce_key)}")
        ]

In [44]:
class NotificationInteractionNode(SQLNode):
    def do_reduce(self, reduce_key):
        return [
            # Shouldn't this just be a select?
            sqlop(optype=SQLOpType.aggfunc, opval=f"count(*) as {self.colabbr('num_interactions')}"),
            sqlop(optype=SQLOpType.agg, opval=f"{self.colabbr(reduce_key)}")
        ]
    
    
    def do_labels(self, reduce_key):
        return [
            sqlop(optype=SQLOpType.aggfunc, opval=f"count(*) as {self.colabbr('had_interaction')}"),
            sqlop(optype=SQLOpType.agg, opval=f"{self.colabbr(reduce_key)}")
        ]

In [45]:
cust = CustNode(fpath='cust',
                prefix='cust',
                client=conn, 
                compute_layer=ComputeLayerEnum.sqlite, 
                columns=['id','name'])

2024-07-05 13:08:42 [warning  ] no `date_key` set for <GraphReduceNode: fpath=cust fmt=>


In [46]:
notif = NotificationNode(fpath='notifications',
                prefix='not',
                client=conn, 
                compute_layer=ComputeLayerEnum.sqlite, 
                columns=['id','customer_id','ts'],
                date_key='ts')

In [47]:
ni = NotificationInteractionNode(fpath='notification_interactions',
                prefix='ni',
                client=conn, 
                compute_layer=ComputeLayerEnum.sqlite, 
                columns=['id','notification_id','interaction_type_id','ts'],
                date_key='ts')

In [48]:
order = OrderNode(
    fpath='orders',
    prefix='ord',
    client=conn,
    compute_layer=ComputeLayerEnum.sqlite,
    columns=['id','customer_id','ts','amount'],
    date_key='ts'
)

In [49]:
oe = OrderEventNode(
    fpath='order_events',
    prefix='oe',
    client=conn,
    compute_layer=ComputeLayerEnum.sqlite,
    columns=['id','order_id','event_id']
)

2024-07-05 13:08:43 [warning  ] no `date_key` set for <GraphReduceNode: fpath=order_events fmt=>


In [50]:
from graphreduce.enum import PeriodUnit

In [57]:
gr = GraphReduce(
    name='sql_dialect_example',
    parent_node=cust,
    label_node=order,
    label_period_unit=PeriodUnit.day,
    label_period_val=30,
    compute_layer=ComputeLayerEnum.sqlite,
    use_temp_tables=True,
    lazy_execution=False
)


In [58]:
gr.add_node(cust)
#gr.add_node(notif)
#gr.add_node(ni)
gr.add_node(order)
#gr.add_node(oe)

In [53]:
gr.add_entity_edge(
    cust,
    notif,
    parent_key='id',
    relation_key='customer_id',
    reduce=True
)

In [54]:
gr.add_entity_edge(
    notif,
    ni,
    parent_key='id',
    relation_key='notification_id',
    reduce=True
)

In [59]:
gr.add_entity_edge(
    cust,
    order,
    parent_key='id',
    relation_key='customer_id',
    reduce=True
)

In [56]:
gr.add_entity_edge(
    order,
    oe,
    parent_key='id',
    relation_key='order_id',
    reduce=True
)

In [60]:
gr.do_transformations_sql()

2024-07-05 13:09:14 [info     ] hydrating graph attributes
2024-07-05 13:09:14 [info     ] hydrating attributes for CustNode
2024-07-05 13:09:14 [info     ] hydrating attributes for OrderNode
2024-07-05 13:09:14 [info     ] checking for prefix uniqueness
2024-07-05 13:09:14 [info     ] no sql was provided for do_normalize so using current data ref
2024-07-05 13:09:14 [info     ] no sql was provided for do_annotate so using current data ref
2024-07-05 13:09:14 [info     ] no sql was provided for do_normalize so using current data ref
2024-07-05 13:09:14 [info     ] depth-first traversal through the graph from source: <GraphReduceNode: fpath=cust fmt=>
2024-07-05 13:09:14 [info     ] reducing relation <GraphReduceNode: fpath=orders fmt=>
2024-07-05 13:09:14 [info     ] joining <GraphReduceNode: fpath=orders fmt=> to <GraphReduceNode: fpath=cust fmt=>
2024-07-05 13:09:14 [info     ] Had label node <GraphReduceNode: fpath=orders fmt=>
2024-07-05 13:09:14 [error    ] no such table: OrderNod

In [75]:
order.get_ref_name(order.do_filters)

'OrderNode_orders_do_filters1'

In [76]:
order._temp_refs

{'do_data': 'OrderNode_orders_do_data',
 'do_annotate': 'OrderNode_orders_do_data',
 'do_filters': 'OrderNode_orders_do_filters',
 'do_normalize': 'OrderNode_orders_do_filters',
 'do_reduce': 'OrderNode_orders_do_reduce',
 'do_labels': 'OrderNode_orders_do_labels'}

In [83]:
pd.read_sql_query("select * from OrderNode_orders_do_filters", conn)

,ord_id,ord_customer_id,ord_ts,ord_amount
0,1,1,2023-05-12,10.0
1,2,1,2023-06-01,11.5
2,3,2,2023-01-01,100.0
3,5,1,2023-07-01,325.0
4,6,2,2023-07-02,23.0
5,7,1,2023-07-14,12000.0


In [ ]:
cust._temp_refs

In [62]:
cust._cur_data_ref

'CustNode_cust_join1'

In [63]:
cust._cur_data_ref

'CustNode_cust_join1'

In [65]:
pd.read_sql_query("select * from CustNode_cust_join1", conn)

,cust_id,cust_name,cust_name_length,ord_customer_id,ord_num_orders,ord_customer_id:1,ord_num_orders:1
0,1,wes,3,1,4,1,4
1,2,john,4,2,2,2,2


In [33]:
pd.read_sql_query("SELECT name FROM sqlite_temp_master WHERE type='table'", conn)

,name
0,CustNode_cust_do_data
1,CustNode_cust_do_annotate
2,CustNode_cust_do_filters
3,NotificationNode_notifications_do_data
4,NotificationNode_notifications_do_annotate
5,NotificationNode_notifications_do_filters
6,NotificationInteractionNode_notification_inter...
7,OrderNode_orders_do_data
8,OrderNode_orders_do_filters
9,OrderEventNode_order_events_do_data


In [32]:
cust._temp_refs

{'do_data': 'CustNode_cust_do_data',
 'do_annotate': 'CustNode_cust_do_annotate',
 'do_filters': 'CustNode_cust_do_filters',
 'do_normalize': 'CustNode_cust_do_filters',
 'join': 'CustNode_cust_join1'}

In [34]:
# time filtering and more is still needed

In [ ]:
# Let's manually execute the customer node and 

In [ ]:
cust.d

In [66]:
isinstance(order, SQLNode)

True

In [74]:
print(order.build_query(order.do_labels('customer_id')))


        SELECT ord_customer_id,
        count(*) as ord_num_orders_label
        FROM OrderNode_orders_do_reduce
        WHERE true
        GROUP BY ord_customer_id
        
